# Running pysentiment (BETO-powered Sentiment Analysis) 

In [1]:
import pandas as pd
import re
import pysentimiento

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
# load dataset with preprocessed text
df = pd.read_csv('dataset_token_ready.csv')

# define regex to search for mentions of president Ortega
regex = r'([Oo]rtega)|([Nn]uestro [Pp]residente)|[Pp]residente de [Nn]icaragua|([Cc]omandante [Dd]aniel)|[Dd]aniel y [Rr]osario'

# subset articles that contain mentions of president Ortega
df = df.loc[df['text'].str.contains(regex, na = False)].reset_index(drop = True)

# split string of texts into list of sentences
df["sentences"] = df.text.apply(lambda x: x.split("."))

# explode rows so that each row contains one sentence
df = df.explode("sentences", ignore_index = True)

# subset sentences that contain mentions of Ortega
df = df.loc[df["sentences"].str.contains(regex)].reset_index(drop = True)

# drop text column
df.drop("text", axis = 1, inplace = True)

<ipython-input-2-ac87f0e27e66>:8: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df.loc[df['text'].str.contains(regex, na = False)].reset_index(drop = True)
<ipython-input-2-ac87f0e27e66>:17: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df.loc[df["sentences"].str.contains(regex)].reset_index(drop = True)


In [3]:
analyzer = pysentimiento.create_analyzer(task="sentiment", lang="es")

In [82]:
analysis = analyzer.predict(sentences_daniel[1])
print(analysis.sentence, "\n\n output:", analysis.output, "\n\n probabilities:",analysis.probas)

"Ese aumento de solicitudes podría significar que el éxodo está empezando pues la gente huye de la cruel represión de Daniel Ortega", valoran los organizados en SOSNicaragua-Madrid, un gran grupo de nicaragüenses que se autoconvocan para realizar actividades de protesta, denuncia, acompañamiento e información, en la capital del país ibérico y que realizó la actividad el pasado 19 de julio 

 output: NEU 

 probabilities: {'NEG': 0.38104572892189026, 'NEU': 0.6105515360832214, 'POS': 0.008402684703469276}


In [5]:
df["sentiment"] = df["sentences"].apply(lambda x: analyzer.predict(x).output)

In [8]:
with open("dataset_sentiment.pkl", "wb") as f:
    pickle.dump(df, f)